In [45]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the preprocessor and the model
preprocessor = joblib.load('preprocessor.joblib')
model = joblib.load('best_model.joblib')
label_encoder = LabelEncoder()

# Initialize the FastAPI app
app = FastAPI()

# Define the input schema for prediction
class PredictionRequest(BaseModel):
    Monthly_Inhand_Salary: float
    Num_Bank_Accounts: float
    Num_Credit_Card: float
    Interest_Rate: float
    Occupation: str
    Credit_Mix: str
    Credit_History_Age: float

# Define the prediction endpoint
@app.post("/predict")
def predict(request: PredictionRequest):
    print(f"Received data: {request}")  # Add a print statement to log the request

    # Convert input data into a DataFrame
    input_data = pd.DataFrame([{
        "Monthly_Inhand_Salary": request.Monthly_Inhand_Salary,
        "Num_Bank_Accounts": request.Num_Bank_Accounts,
        "Num_Credit_Card": request.Num_Credit_Card,
        "Interest_Rate": request.Interest_Rate,
        "Occupation": request.Occupation,
        "Credit_Mix": request.Credit_Mix,
        "Credit_History_Age": request.Credit_History_Age
    }])

    # Perform custom feature engineering (same as in the original experiment)
    input_data['Salary_to_Credit_Card_Ratio'] = input_data['Monthly_Inhand_Salary'] / (input_data['Num_Credit_Card'] + 1)
    input_data['Salary_to_Bank_Account_Ratio'] = input_data['Monthly_Inhand_Salary'] / (input_data['Num_Bank_Accounts'] + 1)

    # Preprocess the input data using the saved preprocessor
    preprocessed_data = preprocessor.transform(input_data)

    # Get the prediction from the model
    prediction = model.predict(preprocessed_data)

    # Reverse label encoding if needed (if target is categorical, like "Good", "Poor")
    prediction_label = label_encoder.inverse_transform(prediction)

    # Return the prediction
    return {"prediction": prediction_label[0]}
